<a href="https://colab.research.google.com/github/rcolo88/Advanced-Mchine-Learning-HW4/blob/main/Robert_Cologero_HW4Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import numpy.linalg as LA
from time import time
import pandas as pd
import os 


Populating the interactive namespace from numpy and matplotlib


$\def\m#1{\mathbf{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$







# Q4: Matrix Factorization for Movie Lens Recommendations (50pt)



### Data preparation (Read carefully!)
In this problem, you will look at a portion of the  movie rating dataset, which
has ratings that users have given to movies. It is a dataset of 1 million movie ratings (from 1 to 5) available from the MovieLens project. The MovieLens datasets were created collected by GroupLens Research at the University of Minnesota.

We want to include the metadata (movie titles, etc), not just the ratings matrix.

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m
!ls
folder = 'ml-1m'

--2022-05-06 12:15:56--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.95MB/s    in 1.4s    

2022-05-06 12:15:58 (3.95 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-1m  ml-1m.zip  sample_data


In [ ]:

ratings_list = [ [int(x) for x in i.strip().split("::")] for i in open(os.path.join(folder,'ratings.dat'), 'r').readlines()]
users_list = [i.strip().split("::") for i in open(os.path.join(folder, 'users.dat'), 'r').readlines()]
movies_list = [i.strip().split("::") for i in open(os.path.join(folder, 'movies.dat'), 'r',  encoding="latin-1").readlines()]

ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [ ]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
def get_movie_name(movies_df, movie_id_str):
  ndx = (movies_df['MovieID']==int(movie_id_str))
  name = movies_df['Title'][ndx].to_numpy()[0]
  return name

def get_movie_genres(movies_df, movie_id_str):
  ndx = (movies_df['MovieID']==int(movie_id_str))
  name = movies_df['Genres'][ndx].to_numpy()[0]
  return name

print('Movie with Moive ID 1 is ', get_movie_name(movies_df, 1))
print('The corresponding genres is ',get_movie_genres(movies_df, 1))
print('Movie with Moive ID 527 is ', get_movie_name(movies_df, "527"))
print('The corresponding genres is ', get_movie_genres(movies_df, "527"))

Movie with Moive ID 1 is  Toy Story (1995)
The corresponding genres is  Animation|Children's|Comedy
Movie with Moive ID 527 is  Schindler's List (1993)
The corresponding genres is  Drama|War


In [ ]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


These look good, but I want the format of my ratings matrix to be one row per user and one column per movie. I'll `pivot` `ratings_df` to get that and call the new variable `R`.

In [ ]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
R = R_df.to_numpy()
R

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [ ]:
print(R.shape)

(6040, 3706)


Similarly we mask a random sample of the user/item ratings to validate and compare how well the recommender system did in predicting the ratings of those masked values. In our case, given we already know each user has given more than 10 ratings, what we'll do is for every user, we remove 10 of the item ratings and and assign them to the test set. 

In [ ]:
def create_train_test(R):
    """
    split into training and test sets,
    remove 5 ratings from each user
    and assign them to the test set
    """
    test = np.zeros(R.shape)
    train = R.copy()
    for user in range(R.shape[0]):
        test_index = np.random.choice(
            np.flatnonzero(R[user]), size = 5, replace = False)

        train[user, test_index] = 0.0
        test[user, test_index] = R[user, test_index]
        
    # assert that training and testing set are truly disjoint
    assert np.all(train * test == 0)
    return train, test

Rtrain, Rtest = create_train_test(R)



---


## Q4.1 SVD (20pt)
The simpliest matrix factorization method is SVD. 

### Your task: 
- Each row(user) subtract by each user mean, which normalize each user rating. 

- Perform SVD on the normalized `R` and choose the latent factors $k=10,30,50$. 

- Calculate the rank=$k$ approximation of $R$. You also need to add the user means back to get the actual ratings prediction, denote this rating matrix as `R_k`

- Now some of entries may be out of rating range $[0,5]$. Please set entries that are larger than 5 to 5 and set entries that are smaller than 0 to 0. 

- Compute MSE for training and testing dataset in all three $k$ situations. Which $k$ performs the best in testing dataset?  


In [ ]:
# Your code starts here
# create normalize function
def normalize(x):
  M,N = x.shape
  out = []
  means = []
  for i in range(M):
    mn = np.mean(x[i])
    means.append(mn)
    new_row = x[i] - mn
    out.extend(new_row)
  out = np.array(out)
  out = np.reshape(out,(M,N))
  return out,np.array(means)

# Define randomized SVD function
def rSVD(X,r,q,p):
    # Step 1: Sample column space of X with P matrix
    N = X.shape[1]
    P = np.random.randn(N,r+p)
    Z = X @ P
    for k in range(q):
        Z = X @ (X.T @ Z)

    Q, R = np.linalg.qr(Z,mode='reduced')

    # Step 2: Compute SVD on projected Y = Q.T @ X
    Y = Q.T @ X
    UY, S, VT = np.linalg.svd(Y,full_matrices=0)

    # Step 3: Compute U
    U = Q @ UY

    return U, S, VT


def MSE(Y,yhat):
  MSE = np.square(np.subtract(Y,yhat)).mean()
  #MSE = 1/Y.shape[1] * ((Y - yhat)**2)
  return MSE

def return_mean(x,means):
  M,N = x.shape
  out = []
  # the following works because M and means are the same size. if not then something is wrong
  for i in range(M):
    new_row = x[i] + means[i]
    out.extend(new_row)
  out = np.array(out)
  out = np.reshape(out,(M,N))
  return out

def adjust(X):
  M,N = X.shape
  x = X.ravel()
  for i in range(len(x)):
    if x[i] < 0:
      x[i] = 0
    elif x[i] > 5:
      x[i] = 5
  x = np.reshape(x,(M,N))
  return x
    
def svd_iterate(X):
  r_s = [10,30,50]
  q = 1
  p =5 
  out = []
  xnorm,mn = normalize(X) 
  for i in r_s:
    U, S, VT= rSVD(xnorm,i,q,p)
    XrSVD = U[:,:(i+1)] @ np.diag(S[:(i+1)]) @ VT[:(i+1),:]
    XrSVD = return_mean(XrSVD,mn)
    XrSVD = adjust(XrSVD)
    out.append(XrSVD)
  x1 = out[0]
  x2 = out[1]
  x3 = out[2]
  return x1, x2, x3




In [ ]:
X1, X3, X5 = svd_iterate(Rtrain)
xt1, xt3, xt5 = svd_iterate(Rtest)

In [ ]:
# This is using my MSE function
print('MSE for k = 10 on training data is ' + str(MSE(Rtrain,X1)))
print('MSE for k = 30 on training data is ' + str(MSE(Rtrain,X3)))
print('MSE for k = 50 on training data is ' + str(MSE(Rtrain,X5)))
print('MSE for k = 10 on testing data is ' + str(MSE(Rtest,xt1)))
print('MSE for k = 30 on testing data is ' + str(MSE(Rtest,xt3)))
print('MSE for k = 50 on testing data is ' + str(MSE(Rtest,xt5)))


MSE for k = 10 on training data is 0.36959840808236577
MSE for k = 30 on training data is 0.33541554218932
MSE for k = 50 on training data is 0.31479872078489923
MSE for k = 10 on testing data is 0.018728950868958753
MSE for k = 30 on testing data is 0.017159724919024278
MSE for k = 50 on testing data is 0.015927022839909936


In [ ]:
# Dr. Ye's MSE 
def compute_mse(train, pred):
        """ignore zero terms prior to comparing the mse"""
        train_ravel = train.ravel()
        pred_ravel  = pred.ravel()
        mask = np.nonzero(train_ravel)
        mse = np.sqrt(np.sum( (train_ravel[mask]- pred_ravel[mask])**2 )/shape(mask)[1])
        return mse

print('MSE for k = 10 on training data is ' + str(compute_mse(Rtrain,X1)))
print('MSE for k = 30 on training data is ' + str(compute_mse(Rtrain,X3)))
print('MSE for k = 50 on training data is ' + str(compute_mse(Rtrain,X5)))
print('MSE for k = 10 on testing data is ' + str(compute_mse(Rtest,xt1)))
print('MSE for k = 30 on testing data is ' + str(compute_mse(Rtest,xt3)))
print('MSE for k = 50 on testing data is ' + str(compute_mse(Rtest,xt5)))

MSE for k = 10 on training data is 2.545593736405239
MSE for k = 30 on training data is 2.3872497581554795
MSE for k = 50 on training data is 2.294855060636084
MSE for k = 10 on testing data is 3.676245623454167
MSE for k = 30 on testing data is 3.479482038470589
MSE for k = 50 on testing data is 3.331340696852087


In [ ]:
r = 50
q = 1
p = 5
normR, meansR = normalize(Rtrain)
U, S, VT= rSVD(normR,r,q,p)
XrSVD = U[:,:(r+1)] @ np.diag(S[:(r+1)]) @ VT[:(r+1),:]
R_k = return_mean(XrSVD,meansR)




### Make Movie Recommendations
Finally, it's time. With the predictions matrix for every user, I can build a function to recommend movies for any user. All I need to do is return the movies with the highest predicted rating that the specified user hasn't already rated. Though I didn't use actually use any explicit movie content features (such as genre or title), I'll merge in that information to get a more complete picture of the recommendations.

I'll also return the list of movies the user has already rated, for the sake of comparison.

In [ ]:
def recommend_movies(preds_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
# R_k with optimal k is the low rank matrix you have constructed previously
preds_df = pd.DataFrame(R_k, columns = R_df.columns)
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,3.165590,0.375669,0.037317,-0.062967,0.085988,0.056262,0.141829,0.149765,0.003277,0.112065,...,-0.058197,0.018268,0.030340,0.013583,-0.114830,0.269235,0.007668,0.009446,0.020686,-0.075230
1,0.654028,0.401195,0.213976,0.089189,0.175418,0.901955,0.029977,0.096926,0.155626,1.414070,...,-0.055898,0.016871,-0.009988,0.081718,-0.012046,0.280287,-0.224011,-0.042023,0.011438,-0.049646
2,2.360325,0.730987,0.203682,-0.125486,0.005817,-0.048460,-0.215464,0.097260,0.040879,0.597988,...,-0.020696,-0.011892,0.019424,0.001611,0.007628,0.037034,-0.221980,-0.037409,0.015232,-0.323243
3,0.199021,0.096959,-0.010939,0.041380,-0.023370,0.083732,-0.129143,0.060036,-0.008680,-0.173434,...,-0.015559,-0.001258,-0.004587,-0.022453,0.000896,-0.004215,-0.085620,-0.026600,0.014638,-0.091829
4,0.750669,-0.149126,-0.260552,0.094818,-0.148139,1.184151,-0.359676,-0.070530,-0.050148,0.555083,...,0.069086,-0.001252,-0.012525,-0.034079,-0.018737,0.240952,0.540043,-0.012636,0.074252,0.299937


In [ ]:
# Let's check for user 837
already_rated, predictions = recommend_movies(preds_df, 837, movies_df, ratings_df, 10)
already_rated


User 837 has already rated 69 movies.
Recommending highest 10 predicted ratings movies not already rated.


,UserID,MovieID,Rating,Timestamp,Title,Genres
36,837,858,5,975360036,"Godfather, The (1972)",Action|Crime|Drama
35,837,1387,5,975360036,Jaws (1975),Action|Horror
65,837,2028,5,975360089,Saving Private Ryan (1998),Action|Drama|War
63,837,1221,5,975360036,"Godfather: Part II, The (1974)",Action|Crime|Drama
11,837,913,5,975359921,"Maltese Falcon, The (1941)",Film-Noir|Mystery
...,...,...,...,...,...,...
29,837,1177,2,975359812,Enchanted April (1991),Drama
57,837,1201,2,975360161,"Good, The Bad and The Ugly, The (1966)",Action|Western
17,837,2289,2,975360869,"Player, The (1992)",Comedy|Drama
7,837,1292,1,975360869,Being There (1979),Comedy


In [ ]:
# here is your prediction. Does it make sense? please comment your result.
predictions

,MovieID,Title,Genres
2085,2194,"Untouchables, The (1987)",Action|Crime|Drama
1848,1953,"French Connection, The (1971)",Action|Crime|Drama|Thriller
1198,1242,Glory (1989),Action|Drama|War
1849,1954,Rocky (1976),Action|Drama
1157,1193,One Flew Over the Cuckoo's Nest (1975),Drama
942,969,"African Queen, The (1951)",Action|Adventure|Romance|War
19,21,Get Shorty (1995),Action|Comedy|Drama
1161,1198,Raiders of the Lost Ark (1981),Action|Adventure
1912,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
47,50,"Usual Suspects, The (1995)",Crime|Thriller


# Your solution:

It appeared k = 50 performed the best, however the results from using k = 10 make sense. We can see for the most part genre's are consistent between action and drama for the already rated and predicted.



---


## Q4.2 Non-negative matrix (30pt)
$W$ and $H$ values in matrix Factorization are non interpretable since their components can take arbitrary (positive and negative) values. 

The objective function of non-negative matrix factorization is 
\begin{align}
\min_{W\in \mb{R}^{M\times k}, H\in \mb{R}^{N\times k}} \|A-WH^\top\|_F^2 + \lambda \left(\|W\|_F^2 + \|H\|_F^2\right), \text{ subject to } W\ge 0, H\ge 0. 
\end{align}

### Your task: 
- Implement Lee and Seung's multiplicative update rule as discussed in the notebook, with the latent factor $k=10$ and $\lambda=0.6$

- Compute MSE for training and testing dataset. 

- Please check your recommendation prediction for the user 837. Does it make sense? 

- Check the matrix $W$, does it fit the preference of genres for the user? You can cross-check some rows.

- Check the matrix $H$, does it fit the description of genres for the movie? You can cross-check some rows.




In [ ]:
# Your code starts here


In [ ]:
class NNM:
    '''A basic rating prediction algorithm based on matrix factorization.'''
    
    def __init__(self, n_factors, n_iter, lamd):
        
        self.n_iter = n_iter  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        self.lam = lamd # regularization parameter
        
    def fit(self, train):
        
        print('Fitting data...')
        self.n_user, self.n_item = train.shape

        
        # Randomly initialize the user and item factors.
        w = np.random.random((self.n_user, self.n_factors))
        h = np.random.random((self.n_item, self.n_factors))
        
        # Lee and Seung's update procedure
        for _ in range(self.n_iter):
          new_W = (w * (np.dot(train,h)))/(self.lam * w + (w @ h.T @ h))  + 0.01
          new_H = (h * (np.dot(new_W.T, train).T))/(self.lam * h + (new_W.T @ new_W @ h.T).T)  +0.01
          w = new_W
          h = new_H
        
        self.W, self.H = w, h
        self.train = train

    def predict(self):
        """predict ratings for every user and item"""
        pred = self.W.dot(self.H.T)
        return pred
        
    def compute_mse(self, train, pred):
        """ignore zero terms prior to comparing the mse"""
        train_ravel = train.ravel()
        pred_ravel  = pred.ravel()
        mask = np.nonzero(train_ravel)
        mse = np.sqrt(np.sum( (train_ravel[mask]- pred_ravel[mask])**2 )/shape(mask)[1])
        return mse



    def validation(self, test):
        """ 
        pass in testing to validate the performance of test data.
        """
        pred = self.predict()
        test_mse = self.compute_mse(test, pred)
        #print(f'MSE for data is {test_mse}')

        return test_mse


In [ ]:
Rtrain.shape

(6040, 3706)

In [ ]:

nnM = NNM(n_factors = 10, n_iter = 100, lamd = 0.6)
nnM.fit(Rtrain)

Fitting data...


In [ ]:
print(f"MSE for training set is {nnM.validation(Rtrain)}")
print(f"MSE for testing set is {nnM.validation(Rtest)}")

MSE for training set is 2.61495148046342
MSE for testing set is 3.107838070073692


We run the code below out of the class in order to recover the w and h matrices later on.

In [ ]:
n_user, n_item = Rtrain.shape
n_iter= 100
lam =0.6
n_factors = 10
# Randomly initialize the user and item factors.
w = np.random.random((n_user, n_factors))
h = np.random.random((n_item, n_factors))
        
# Lee and Seung's update procedure
for _ in range(n_iter):
  new_W = (w * (np.dot(Rtrain,h)))/(lam * w + (w @ h.T @ h)) + 0.01
  new_H = (h * (np.dot(new_W.T, Rtrain).T))/(lam * h + (new_W.T @ new_W @ h.T).T) + 0.01
  w = new_W
  h = new_H
        

In [ ]:
predicted = np.dot(w,h.T)

In [ ]:
def compute_mse(train, pred):
        """ignore zero terms prior to comparing the mse"""
        train_ravel = train.ravel()
        pred_ravel  = pred.ravel()
        mask = np.nonzero(train_ravel)
        mse = np.sqrt(np.sum( (train_ravel[mask]- pred_ravel[mask])**2 )/shape(mask)[1])
        return mse

print(f"MSE on the train set using given MSE function is {compute_mse(Rtrain,predicted)}")
print(f"MSE on the train set using my MSE function is {MSE(Rtrain,predicted)}")

MSE on the train set using given MSE function is 2.6129504046529988
MSE on the train set using my MSE function is 0.3928078736427732


In [ ]:
from pandas.core.frame import DataFrame
predicteddf = DataFrame(predicted, columns = R_df.columns)
predicteddf

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1.457443,0.719728,0.283195,0.103338,0.279992,0.230014,0.268830,0.110985,0.029104,0.265028,...,0.030759,0.012968,0.023476,0.028117,0.021685,0.231892,0.085006,0.030636,0.019634,0.100838
1,1.202197,0.482427,0.275904,0.105466,0.208473,1.423467,0.339872,0.042127,0.210553,1.556578,...,0.053381,0.018439,0.023024,0.066552,0.049785,0.401370,0.133544,0.049532,0.031657,0.199557
2,0.847228,0.190268,0.118140,0.043135,0.088783,0.377125,0.105621,0.026636,0.046736,0.323560,...,0.026692,0.009958,0.013874,0.025906,0.020984,0.207987,0.074832,0.025963,0.015795,0.093721
3,0.418481,0.105465,0.044517,0.019256,0.032037,0.184597,0.045777,0.013258,0.020086,0.134025,...,0.016051,0.005510,0.007931,0.013392,0.015882,0.105610,0.047190,0.016264,0.009455,0.057273
4,1.046225,0.239098,0.210064,0.162991,0.126507,1.039160,0.193877,0.050333,0.058241,0.244778,...,0.146296,0.022010,0.027901,0.081929,0.057427,0.916801,0.836032,0.122054,0.086197,0.723036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,2.795000,1.242954,0.707219,0.495921,0.528042,2.169291,0.852703,0.202765,0.176374,0.918663,...,0.385006,0.072618,0.092456,0.198546,0.252929,1.669571,1.817880,0.334686,0.231699,1.605755
6036,1.903280,0.242370,0.129017,0.070868,0.089939,0.680685,0.140747,0.054085,0.053905,0.281295,...,0.117087,0.030362,0.035033,0.057760,0.115810,0.485439,0.242215,0.092444,0.061225,0.292353
6037,0.237248,0.064499,0.055038,0.026707,0.042617,0.080765,0.089026,0.010034,0.010608,0.067820,...,0.012180,0.003398,0.004394,0.007996,0.009815,0.079458,0.037231,0.010254,0.007236,0.039394
6038,1.051638,0.514222,0.223842,0.093363,0.210933,0.140843,0.249152,0.087963,0.034946,0.246989,...,0.075911,0.020213,0.025675,0.035181,0.078306,0.194519,0.120047,0.055642,0.039470,0.113720


In [ ]:
already_rated, predictions = recommend_movies(predicteddf, 837, movies_df, ratings_df, 10)


User 837 has already rated 69 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [ ]:
already_rated.head(10)

,UserID,MovieID,Rating,Timestamp,Title,Genres
36,837,858,5,975360036,"Godfather, The (1972)",Action|Crime|Drama
35,837,1387,5,975360036,Jaws (1975),Action|Horror
65,837,2028,5,975360089,Saving Private Ryan (1998),Action|Drama|War
63,837,1221,5,975360036,"Godfather: Part II, The (1974)",Action|Crime|Drama
11,837,913,5,975359921,"Maltese Falcon, The (1941)",Film-Noir|Mystery
20,837,3417,5,975360893,"Crimson Pirate, The (1952)",Adventure|Comedy|Sci-Fi
34,837,2186,4,975359955,Strangers on a Train (1951),Film-Noir|Thriller
55,837,2791,4,975360893,Airplane! (1980),Comedy
31,837,1188,4,975360920,Strictly Ballroom (1992),Comedy|Romance
28,837,1304,4,975360058,Butch Cassidy and the Sundance Kid (1969),Action|Comedy|Western


In [ ]:
predictions

,MovieID,Title,Genres
581,593,"Silence of the Lambs, The (1991)",Drama|Thriller
596,608,Fargo (1996),Crime|Drama|Thriller
516,527,Schindler's List (1993),Drama|War
47,50,"Usual Suspects, The (1995)",Crime|Thriller
253,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2648,2762,"Sixth Sense, The (1999)",Thriller
1159,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1161,1198,Raiders of the Lost Ark (1981),Action|Adventure
1157,1193,One Flew Over the Cuckoo's Nest (1975),Drama
888,912,Casablanca (1942),Drama|Romance|War


In [ ]:
print('Genres user 837 likes have a higher rating')
print(w[837])
print('\n')
print('Second movie genre weightings')
print(h[1])
print('\n')
print('Predicted rating')
print(w[837].dot(h[1].T))
print('\n')
print('First movie genre weightings')
print(h[0])
print('\n')
print('Predicted rating')
print(w[837].dot(h[0].T))


Genres user 837 likes have a higher rating
[0.01759128 0.56119    0.01543865 0.02759019 0.11300692 0.03916346
 0.99442688 0.04027331 0.02072136 0.06250441]


Second movie genre weightings
[1.3706406  0.26141511 0.08900584 0.57945829 0.08015229 0.04383801
 0.20260224 0.07525554 0.13008805 0.04576654]


Predicted rating
0.4090110943447955


First movie genre weightings
[1.82463925 0.63127399 0.76883503 0.32639723 0.08149127 0.26943197
 0.23801811 1.79860543 0.11122948 1.66333271]


Predicted rating
0.8423963496653192


# Your solution:
We can see the predicted movies' genres match up with the already rated pretty well. As for the W and H matrices we can see some interesting things. User 837 appears to rate 2 genres very high which are in the second and first spot in the above array. From the H matrix we can see any movie that has a high value in these matching positions will have a higher recomended rating score for the given user. 